In [1]:
import tensorflow as tf
from tensorflow import keras

from keras.layers import Input, Dense, Activation, Dropout
from keras.models import Sequential
from keras.models import Model

import numpy as np
import matplotlib.pyplot as plt
from keras.optimizers import Adam
import pandas as pd

C:\Users\tsmar\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#the first thing we need to do is get the data and the labels for the data. 
DataFrameMain = pd.read_csv("C:/Users/tsmar/election_dataframe.csv")
DataFrame = DataFrameMain.select_dtypes(include=["int", "float"])
headers = list(DataFrame.columns.values)

#calculate the split size based on percent of data for train and test
lenTrain = int(0.75*len(DataFrame))
lenTest = int(0.25*len(DataFrame))

#create a mean GOP column, and a mean DEM column (dont worry not inputs into actual training session)
DataFrame["meanGOP"] = DataFrame[["Republicans 2016", "Republicans 2012", "Republicans 2008"]].mean(axis=1) / 100
DataFrame["meanDEM"] = DataFrame[["Democrats 2016", "Democrats 2012", "Democrats 2008"]].mean(axis=1) / 100

#now lets normalize the data so it can actually be used. 
DataFrame["Percent of adults with a bachelor's degree or higher, 2012-2016"] = DataFrame["Percent of adults with a bachelor's degree or higher, 2012-2016"] / 100
DataFrame["Percent of adults completing some college or associate's degree, 2012-2016"] = DataFrame["Percent of adults completing some college or associate's degree, 2012-2016"] / 100
DataFrame["Percent of adults with a high school diploma only, 2012-2016"] = DataFrame["Percent of adults with a high school diploma only, 2012-2016"] / 100
DataFrame["Percent of adults with less than a high school diploma, 2012-2016"] = DataFrame["Percent of adults with less than a high school diploma, 2012-2016"]/100

#more normalization of fields
DataFrame['Median Earnings 2010'] = DataFrame['Median Earnings 2010'] / 100000
DataFrame['White Population'] = DataFrame['White Population'] / 100
DataFrame['African American Population'] = DataFrame['African American Population'] / 100
DataFrame['Native American Population'] = DataFrame['Native American Population'] / 100
DataFrame['Asian American Population'] = DataFrame['Asian American Population'] / 100
DataFrame['Other Race or Races'] = DataFrame['Other Race or Races'] / 100
DataFrame['Latino Population'] = DataFrame['Latino Population'] / 100
DataFrame['Children Under 6 Living in Poverty'] = DataFrame['Children Under 6 Living in Poverty'] /100
DataFrame['Adults 65 and Older Living in Poverty'] = DataFrame['Adults 65 and Older Living in Poverty'] /100
DataFrame['Total Population'] = DataFrame['Total Population'] / 1000000
DataFrame['Preschool Enrollment'] = DataFrame['Preschool Enrollment'] / 100
DataFrame['Poverty Rate'] = DataFrame['Poverty Rate'] / 100
DataFrame['Professional Occupations'] = DataFrame['Professional Occupations'] / 100
DataFrame['Service Occupations'] = DataFrame['Service Occupations'] / 100
DataFrame['Sales and Office Occupations'] = DataFrame['Sales and Office Occupations'] / 100
DataFrame['Sales and Office Occupations'] = DataFrame['Sales and Office Occupations'] / 100
DataFrame['Farming, Fishing, and Forestry Occupations'] = DataFrame['Farming, Fishing, and Forestry Occupations'] /100
DataFrame['Construction Occupations'] = DataFrame['Construction Occupations'] /100
DataFrame['Construction Occupations'] = DataFrame['Construction Occupations'] /100
DataFrame['Manufacturing and Transportation Occupations'] = DataFrame['Manufacturing and Transportation Occupations'] / 100
DataFrame['Median Age'] = DataFrame['Median Age'] /100

#pull out relevant fields for training and testing
trainData = DataFrame[["meanGOP", "meanDEM", "Percent of adults with a bachelor's degree or higher, 2012-2016",
                     "Percent of adults completing some college or associate's degree, 2012-2016",
                     "Percent of adults with a high school diploma only, 2012-2016",
                    "Percent of adults with less than a high school diploma, 2012-2016",
                     'Median Earnings 2010','White Population','African American Population',
                     'Native American Population','Asian American Population','Other Race or Races',
                     'Latino Population', 'Children Under 6 Living in Poverty', 'Adults 65 and Older Living in Poverty',
                     'Total Population', 'Preschool Enrollment', 'Poverty Rate', 
                      'Service Occupations', 'Sales and Office Occupations', 'Farming, Fishing, and Forestry Occupations', 
'Construction Occupations', 'Manufacturing and Transportation Occupations','Racial Homogenity','Median Age']][:lenTrain]

testData = DataFrame[["meanGOP", "meanDEM", "Percent of adults with a bachelor's degree or higher, 2012-2016",
                     "Percent of adults completing some college or associate's degree, 2012-2016",
                     "Percent of adults with a high school diploma only, 2012-2016",
                    "Percent of adults with less than a high school diploma, 2012-2016",
                     'Median Earnings 2010','White Population','African American Population',
                     'Native American Population','Asian American Population','Other Race or Races',
                     'Latino Population', 'Children Under 6 Living in Poverty', 'Adults 65 and Older Living in Poverty',
                     'Total Population', 'Preschool Enrollment', 'Poverty Rate',
                     'Service Occupations', 'Sales and Office Occupations', 'Farming, Fishing, and Forestry Occupations', 
'Construction Occupations', 'Manufacturing and Transportation Occupations','Racial Homogenity', 'Median Age']][-1*lenTest:]

#'Median Age', 'Sick Days', 'Mental Health Days', 'Low Birthweight', 'Teen Birth Rate', 'Children in Single Parent Households', 'Adult Smoking', 'Adult Obesity',
trainLabels = []
testLabels = []
#build the train Labels
for index in range(0, len(trainData["meanGOP"])):
    rawData = trainData.values
    j = rawData[index][0]
    k = rawData[index][1]
    if j > k:
        trainLabels.append(np.array([1, 0])) #gop
    else:
        trainLabels.append(np.array([0, 1])) #dem
    
#build the test labels
for index in range(0, len(testData["meanGOP"])):
    rawData = testData.values
    j = rawData[index][0]
    k = rawData[index][1]
    if j > k:
        testLabels.append(np.array([1, 0])) #gop
    else:
        testLabels.append(np.array([0, 1])) #dem


#build the np arrays for passing to keras. 
x_train = np.array(trainData[["Percent of adults with a bachelor's degree or higher, 2012-2016",
                     "Percent of adults completing some college or associate's degree, 2012-2016",
                     "Percent of adults with a high school diploma only, 2012-2016",
                    "Percent of adults with less than a high school diploma, 2012-2016",
                     'Median Earnings 2010','White Population','African American Population',
                     'Native American Population','Asian American Population','Other Race or Races',
                     'Latino Population', 'Children Under 6 Living in Poverty', 'Adults 65 and Older Living in Poverty',
                     'Total Population', 'Preschool Enrollment', 'Poverty Rate','Service Occupations', 'Sales and Office Occupations', 'Farming, Fishing, and Forestry Occupations', 
'Construction Occupations', 'Manufacturing and Transportation Occupations','Racial Homogenity', 'Median Age']].values, dtype='float64')
y_train = np.array(trainLabels)

x_test = np.array(testData[["Percent of adults with a bachelor's degree or higher, 2012-2016",
                     "Percent of adults completing some college or associate's degree, 2012-2016",
                     "Percent of adults with a high school diploma only, 2012-2016",
                    "Percent of adults with less than a high school diploma, 2012-2016",
                     'Median Earnings 2010','White Population','African American Population',
                     'Native American Population','Asian American Population','Other Race or Races',
                     'Latino Population', 'Children Under 6 Living in Poverty', 'Adults 65 and Older Living in Poverty',
                     'Total Population', 'Preschool Enrollment', 'Poverty Rate','Service Occupations', 'Sales and Office Occupations', 'Farming, Fishing, and Forestry Occupations', 
'Construction Occupations', 'Manufacturing and Transportation Occupations','Racial Homogenity', 'Median Age']].values, dtype='float64')
y_test = np.array(testLabels)


print(x_train[:10], y_train[:10])




C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

[[2.45930000e-01 2.86600000e-01 3.43310000e-01 1.24170000e-01
  3.00432137e-01 7.75500000e-01 1.77000000e-01 4.00000000e-03
  7.50000000e-03 1.35000000e-02 2.20000000e-02 2.11500000e-01
  8.75000000e-02 5.20770000e-02 4.23000000e-01 1.04500000e-01
  1.63500000e-01 2.89000000e-03 5.00000000e-03 1.12000000e-03
  1.35500000e-01 6.33467750e-01 3.62000000e-01]
 [2.95470000e-01 3.17880000e-01 2.86920000e-01 9.97200000e-02
  2.66305654e-01 8.43000000e-01 9.55000000e-02 6.00000000e-03
  6.50000000e-03 1.25000000e-02 3.65000000e-02 2.09000000e-01
  5.20000000e-02 1.77131000e-01 4.30000000e-01 1.20500000e-01
  1.74000000e-01 2.74000000e-03 8.50000000e-03 1.23500000e-03
  1.09000000e-01 7.21336000e-01 4.10000000e-01]
 [1.28680000e-01 2.59690000e-01 3.49270000e-01 2.62360000e-01
  2.06634527e-01 4.74000000e-01 4.65000000e-01 3.50000000e-03
  2.50000000e-03 9.00000000e-03 4.70000000e-02 3.71500000e-01
  2.13000000e-01 2.85600000e-02 3.25000000e-01 2.48500000e-01
  1.49000000e-01 2.01000000e-03 2.05

C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
